In [34]:
import pandas as pd
import numpy as np
import scipy.sparse
import csv
import sys
from scipy.sparse import csr_matrix

In [2]:
import pickle
import timeit
def save_obj(obj, name ):
    with open('dumps/'+name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('dumps/'+name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
csv.field_size_limit(sys.maxsize)
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [4]:
f = open('user.csv', 'r')
reader = csv.reader(f)
reader = csv.DictReader(f)
user_id_list = []
for i,row in enumerate(reader):
        user_id_list.append(row['user_id'])

In [5]:
print len(set(user_id_list)),len(user_id_list)

1326101 1326101


In [6]:
f = open('business.csv', 'r')
reader = csv.reader(f)
reader = csv.DictReader(f)
business_id_list = []
for i,row in enumerate(reader):
        business_id_list.append(row['business_id'])

In [7]:
print len(set(business_id_list)),len(business_id_list)

174567 174567


In [8]:
save_obj(business_id_list, 'business_id_list' )
save_obj(user_id_list, 'user_id_list' )

In [9]:
def itemize(data,s):
#     _,indx = np.unique(data,return_index=True)
#     u = data[np.sort(indx)]
    u = load_obj(s)
    n_data = len(u)
    new_indx = np.arange(n_data)
    d = dict(zip(u,new_indx))
    data_indx = np.zeros(data.shape, dtype=np.int32)
    for i in range(data_indx.shape[0]):
        data_indx[i] = d[data[i]]
    return data_indx, n_data

def load_data():
    data = pd.read_csv('review.csv')
    data = data.drop(['funny', 'review_id', 'text', 'date', 'useful', 'cool'], axis=1)
    data.drop_duplicates(inplace=True)
    rows, cols, stars = np.array(data['user_id']), np.array(data['business_id']), np.array(data['stars'],dtype=np.uint8)
    # itemize users and items
    row_indx, n_users = itemize(rows,'user_id_list')
    col_indx, n_items = itemize(cols,'business_id_list' )
    return scipy.sparse.csr_matrix((stars,(row_indx, col_indx)), dtype=np.uint8, shape=(n_users,n_items))

# R = scipy.sparse.csr_matrix((stars,(row_indx, col_indx)), dtype=np.uint8, shape=(n_users,n_items))
R = load_data()

In [10]:
tr_list = R.nonzero()
No_entries = len(tr_list[0])
print (No_entries)

5261667


In [11]:
R.shape  # (users, businesses) 

(1326101, 174567)

In [12]:
import random
import copy

In [13]:
cold_start_users = random.sample(range(R.shape[0]),int(R.shape[0]*0.025))
cold_start_items = random.sample(range(R.shape[1]),int(R.shape[1]*0.025))

In [14]:
cold_start_users = set(cold_start_users)
cold_start_items = set(cold_start_items)

In [15]:
R_copy = copy.deepcopy(R)

In [16]:
cold_start_user_triplet = []
cold_start_item_triplet = []

In [17]:
print (len(tr_list[0]))
total = len(tr_list[0])

5261667


In [18]:
print (len(cold_start_users),len(cold_start_items))

(33152, 4364)


In [19]:
for i in zip(tr_list[0],tr_list[1]):
    if i[0] in cold_start_users:
        cold_start_user_triplet.append(i)
        R_copy[i[0],i[1]] = 0
    if i[1] in cold_start_items:
        cold_start_item_triplet.append(i)
        R_copy[i[0],i[1]] = 0

In [20]:
train_warmTest_TMP = R_copy.nonzero()
print (len(train_warmTest_TMP[0]))
train_warmTest = [(i[0],i[1]) for i in zip(train_warmTest_TMP[0],train_warmTest_TMP[1])]
warm_test = random.sample(train_warmTest,200000)
for i in warm_test:
    R_copy[i[0],i[1]] = 0

train = R_copy.nonzero()

train = [(i[0],i[1]) for i in zip(train[0],train[1])]

print (len(train))

5001323
4801323


In [21]:
print ('train',len(train))
print ('warm_test',len(warm_test))
print ('cold_item',len(cold_start_item_triplet))
print ('cold_user',len(cold_start_user_triplet))

('train', 4801323)
('warm_test', 200000)
('cold_item', 129248)
('cold_user', 134378)


In [28]:
with open('files/test_cold_item.csv','w') as f:
    for i in cold_start_item_triplet:
        f.write(   str(i[0]+1)+','+str(i[1]+1)+','+str(R[i[0],i[1]])+ ','+ str(random.randint(1,400000))+'\n')

In [29]:
with open('files/test_cold_user.csv','w') as f:
    for i in cold_start_user_triplet:
        f.write(   str(i[0]+1)+','+str(i[1]+1)+','+str(R[i[0],i[1]])+ ','+ str(random.randint(1,400000))+'\n')

In [30]:
with open('files/test_warm.csv','w') as f:
    for i in warm_test:
        f.write(   str(i[0]+1)+','+str(i[1]+1)+','+str(R[i[0],i[1]])+ ','+ str(random.randint(1,400000))+'\n')

In [31]:
with open('files/train.csv','w') as f:
    for i in train:
        f.write(   str(i[0]+1)+','+str(i[1]+1)+','+str(R[i[0],i[1]])+ ','+ str(random.randint(1,400000))+'\n')

In [32]:
with open('files/test_cold_item_item_ids.csv','w') as f:
    for i in cold_start_item_triplet:
        f.write(str(i[1]+1)+'\n')
        
with open('files/test_cold_user_item_ids.csv','w') as f:
    for i in cold_start_user_triplet:
        f.write(str(i[1]+1)+'\n')
        
with open('files/test_warm_item_ids.csv','w') as f:
    for i in warm_test:
        f.write(str(i[1]+1)+'\n')

In [33]:
save_obj(warm_test,'warm_test')
save_obj(cold_start_item_triplet,'cold_item')
save_obj(cold_start_user_triplet,'cold_user')
save_obj(train,'train')
save_obj(R_copy,'R_train')
save_obj(R,'R_full')

KeyboardInterrupt: 

In [35]:
lines = open('files/train.csv').readlines()

In [38]:
x = []
y = []
rat = []
for line in lines:
    line_parts = line.split(',')
    x.append(int(line_parts[0])-1)
    y.append(int(line_parts[1])-1)
    rat.append(int(line_parts[2]))

In [40]:
R_train = csr_matrix((rat,(x, y)), dtype=np.uint8, shape=(1326101, 174567))

In [43]:
print len(R_train.nonzero()[0]),len(x)

4801323 4801323
